## Pipeline

### Imports

In [2]:
from pydantic.v1 import BaseSettings, Field
from pathlib  import Path
import pandas as pd
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from uuid import uuid4
from langchain_mistralai import ChatMistralAI
from google.colab import userdata
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda, Runnable
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from typing import Any
from itertools import zip_longest
from langchain_community.vectorstores.weaviate import Weaviate
import weaviate
from weaviate import EmbeddedOptions


from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase
from deepeval.test_case import LLMTestCaseParams
from deepeval.dataset import EvaluationDataset

from dotenv import find_dotenv, load_dotenv
from deepeval.test_case import LLMTestCase
import math
from langchain_ollama.chat_models import ChatOllama

In [3]:
_ = load_dotenv(find_dotenv())
import os
os.environ['OPENAI_API_KEY'] = userdata.get('openai')

In [4]:
OPENAI_MODEL = 'gpt-4o-mini'

### Templates

In [5]:
GENERATION_TEMPLATE = """
Ответь на вопрос, используя контекст:

КОНТЕКСТ:
=====
{context}
=====

ВОПРОС:
=====
{question}
=====
""".strip()

### Settings

In [6]:
class DefaultSettings(BaseSettings):
  project_path: Path = Path("/content/sample_data")
  data_path: Path = project_path / "data"
  interim_data_path: Path = data_path / "actual/interim"
  data_for_db_path: Path = interim_data_path / "data_for_db.parquet"
  evaluation_df_path: Path = interim_data_path / "evaluation_df.parquet"
  vectorstore_data_path: Path = data_path / "chroma_vectorstore"
  db_collection_name: str = "Rutube"


In [7]:
settings = DefaultSettings()

### Functions

In [8]:
def load_embedding_model():
  model_name = "intfloat/multilingual-e5-large"
  model_kwargs = {"device": "cuda"}
  encode_kwargs = {"normalize_embeddings": True}
  hf = HuggingFaceBgeEmbeddings(
      model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
  )
  return hf

def load_chat_model():
  chat_model = ChatMistralAI(
    model='mistral-small-latest',
    temperature=0,
    api_key=userdata.get('mistral')
  )
  return chat_model

In [9]:
# def load_chat_model():
#   chat_model = ChatMistralAI(
#     model='mistral-small-latest',
#     temperature=0,
#     api_key=userdata.get('mistral')
#   )
#   return chat_model

In [10]:
def load_chat_model():
#   llm = VLLMOpenAI(
#     openai_api_key="EMPTY",
#     openai_api_base="http://89.169.135.235:8000/v1",
#     model_name="mistralai/Mistral-Nemo-Instruct-2407",
#   )
#   return llm

  llm = ChatOllama(model="mistral-nemo", base_url="http://89.169.135.235:11434")

  return llm

In [11]:
# import httpx
# import asyncio
# import time

# from langchain_community.llms import VLLMOpenAI

# def load_chat_model():
#   llm = VLLMOpenAI(
#     openai_api_key="EMPTY",
#     openai_api_base="http://89.169.135.235:8000/v1",
#     model_name="mistralai/Mistral-Nemo-Instruct-2407",
#   )
#   return llm

# # url = "http://89.169.135.235:8000/v1/chat/completions"
# # headers = {"Content-Type": "application/json", "Authorization": "Bearer token"}


In [12]:
# import httpx
# import asyncio
# import time

# from langchain_nvidia_ai_endpoints import ChatNVIDIA

# def load_chat_model():


#   # connect to an embedding NIM running at localhost:8000, specifying a specific model
#   llm = ChatNVIDIA(base_url="http://89.169.135.235:8000/v1", model="mistralai/Mistral-Nemo-Instruct-2407")
#   return llm


In [13]:
chat_m = load_chat_model()
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)
chain = chat_template | chat_m
chain.invoke({'name': 'Tom', 'user_input': "Расскажи анекдот про имя свое/?"})


AIMessage(content="Well, here's one for you:\n\nWhy don't I ever get lost?\n\nBecause my name is Tom - it's always ahead of me!", additional_kwargs={}, response_metadata={'model': 'mistral-nemo', 'created_at': '2024-09-28T11:23:24.855277098Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4449925956, 'load_duration': 3686335044, 'prompt_eval_count': 45, 'prompt_eval_duration': 65349000, 'eval_count': 30, 'eval_duration': 519081000}, id='run-4c73aea5-5e33-42fa-b2d6-833270ef1074-0', usage_metadata={'input_tokens': 45, 'output_tokens': 30, 'total_tokens': 75})



```
# Выбран кодовый формат
```

chat_m = load_chat_model()
chat_m.invoke("Как дела?")

In [14]:
# db._client.delete_collection('rutube')

In [15]:
# query = db._client. query  # type: ignore
# response = query.aggregate(settings.db_collection_name).with_meta_count().do()
# rows_count = response["data"]["Aggregate"][settings.db_collection_name][0]["meta"]["count"]
# print(rows_count)

In [16]:
def load_data(settings):
  data_for_db = pd.read_parquet(settings.data_for_db_path)
  data_for_db['id'] = data_for_db.index
  return data_for_db

In [17]:
def create_vectorstore(data_for_db, settings: DefaultSettings):


  hf = load_embedding_model()

  # db = Chroma(
  #   collection_name='rutube_hack',
  #   embedding_function=hf,
  #   persist_directory=settings.vectorstore_data_path
  # )
  client = weaviate.Client(
        embedded_options=EmbeddedOptions(persistence_data_path=settings.vectorstore_data_path)
    )

  if client.schema.exists(settings.db_collection_name):
      client.schema.delete_class(settings.db_collection_name)
  class_obj = {"class": settings.db_collection_name, "vectorIndexConfig": {"distance": "cosine"}}
  client.schema.create_class(class_obj)

  db = Weaviate(
        client=client,
        index_name=settings.db_collection_name,
        text_key="text",
        embedding=hf,
        attributes=["answer", "first_class", "second_class", "id_"],
        by_text=False,
    )

  response = client.query.aggregate(settings.db_collection_name).with_meta_count().do()
  print(f"Number of documents in database: {response}")

  # Вопрос из БЗ	Ответ из БЗ	Классификатор 1 уровня	Классификатор 2 уровня
  documents = []
  for _, row in data_for_db.iterrows():
      documents.append(Document(
          page_content=row['Вопрос из БЗ'],
          metadata={'answer':row['Ответ из БЗ'],'first_class':row['Классификатор 1 уровня'],'second_class':row['Классификатор 2 уровня'],'id_':row['id']}

      ))

  uuids = [str(uuid4()) for _ in range(len(documents))]

  db.add_documents(documents=documents, ids=uuids)

  return db

In [18]:
def create_retriever(db):
  retriever = db.as_retriever(
    search_type='similarity',
    search_kwargs={'k':10}
  )
  return retriever

In [19]:
def format_docs(docs):
    return "\n\n".join([(doc.page_content + " " + doc.metadata['answer']) for doc in docs])

In [20]:
def create_simple_chain_with_sources(db) -> Runnable:

  response = db._client.query.aggregate(settings.db_collection_name).with_meta_count().do()
  rows_count = response["data"]["Aggregate"][settings.db_collection_name][0]["meta"]["count"]
  print(rows_count)

  retriever = db.as_retriever(
    search_type='similarity',
    search_kwargs={'k':10}
  )

  rag_prompt = ChatPromptTemplate.from_messages(
    ('user', GENERATION_TEMPLATE)

  )
  output_parser = StrOutputParser()

  model = load_chat_model()

  map_params: dict[str, Any] = {
          "question": itemgetter("question"),
          "context": itemgetter("context") | RunnableLambda(format_docs),
  }
  chain_without_source: Runnable = RunnableParallel(map_params) | rag_prompt | model | output_parser

  # chain_without_source = rag_prompt | model | output_parser
  source_map_params: dict[str, Any] = {
      "context": itemgetter("question") | retriever,
      "question": itemgetter("question"),
      # "all_documents": itemgetter("question") | retriever,
  }
  chain_with_source = RunnableParallel(source_map_params).assign(answer=chain_without_source)

  return chain_with_source

In [21]:
def generate_answer(question: str, chain: Runnable) -> dict[str, Any]:
  response = chain.invoke({"question": question})
  return response

chain = create_simple_chain_with_sources(db)

question = "что такое рутуб"
generate_answer(question, chain)

In [22]:
def generate_answers_invoke(questions: list[str], chain: Runnable) -> list[dict[str, Any]]:
  responses = []
  for question in questions:
    responses.append(chain.invoke(
          {"question": question}
      )
    )
    # time.sleep(1)
  return responses

In [23]:
def create_dataset_for_metrics(llm_answers_info: dict) -> pd.DataFrame:
  return pd.DataFrame({'input': [llm_answer_info['question'] for llm_answer_info in llm_answers_info],
                       'actual_output': [llm_answer_info['answer'] for llm_answer_info in llm_answers_info],
                       'actual_context_to_llm': [format_docs(llm_answer_info['context']).split('\n\n') for llm_answer_info in llm_answers_info],
                       'actual_context_id': [[doc.metadata['id_'] for doc in llm_answer_info['context']] for llm_answer_info in llm_answers_info],
                       },
                      )

In [64]:
def calculate_relevance(row):
  relevance_metric = GEval(
    model=OPENAI_MODEL,
    name="Document Relevance",
    strict_mode=True,  # Enforce strict binary scoring
    evaluation_steps=[
        "Check if the 'retrieval_context' contains documents that are directly relevant to the 'input'. Penalize if the documents are not related to the user's question.",
        "Verify that key terms or concepts from the 'input' are represented in the 'retrieval_context'. Heavily penalize if any critical information is missing.",
        "Ensure that the 'retrieval_context' covers all necessary facts or details to fully answer the 'input'. Penalize for omissions of important information.",
        "Allow for minor irrelevant content in the 'retrieval_context', but penalize if it significantly distracts from the relevance to the 'input'."
    ],
    evaluation_params=[
        LLMTestCaseParams.INPUT,  # The user's question or query
        LLMTestCaseParams.RETRIEVAL_CONTEXT  # The retrieved documents to check for relevance
    ]
  )
  relevance_metric_test_case = LLMTestCase(
    input=row['input'],
    actual_output=row['actual_output'],
    retrieval_context=row['actual_context_to_llm'],
  )
  relevance_metric.measure(relevance_metric_test_case)
  return relevance_metric.score#, relevance_metric.reason

In [65]:
def calculate_relevance_metric(df_with_metrics):
  df_with_metrics['relevance_metric_score'] = df_with_metrics.apply(calculate_relevance, axis=1)

  return df_with_metrics

In [67]:
def calculate_answer_relevance(row):
  answer_relevance_metric = GEval(
      model=OPENAI_MODEL,
      name="Answer Relevance and Completeness",
      strict_mode=True,  # Enforce strict binary scoring
      evaluation_steps=[
          "Check if the 'actual_output' directly addresses the main point(s) of the 'input'. Penalize if the answer is off-topic or misses the core question.",
          "Verify whether the facts in the 'actual_output' align with the 'input' and any provided context. Penalize if any factual contradictions are present.",
          "Heavily penalize omission of critical details from the 'actual_output'. Ensure that all relevant parts of the 'input' are addressed.",
          "Minor vagueness or unnecessary information can be tolerated, but penalize if it confuses the user or detracts from the clarity of the response."
      ],
      evaluation_params=[
          LLMTestCaseParams.INPUT,  # The user's question or query
          LLMTestCaseParams.ACTUAL_OUTPUT  # The generated answer to evaluate
      ]
  )

  answer_relevance_metric_test_case = LLMTestCase(
    input=row['input'],
    actual_output=row['actual_output'],
  )

  answer_relevance_metric.measure(answer_relevance_metric_test_case)
  return answer_relevance_metric.score#, answer_relevance_metric.reason

In [68]:
def calculate_answer_relevance_metric(df_with_metrics):

  df_with_metrics['answer_relevance_metric_score'] = df_with_metrics.apply(calculate_answer_relevance, axis=1)

  return df_with_metrics

In [69]:
def calculate_hallucination_detection(row):
  hallucination_detection_metric = GEval(
        model=OPENAI_MODEL,
        name="Hallucination Detection",
        strict_mode=True,
        evaluation_steps=[
            "Check if all factual information in the 'actual_output' is grounded in the 'retrieval_context'. Penalize if any information is introduced that is not supported by the retrieved documents.",
            "Heavily penalize if key facts in the 'actual_output' are fabricated or not present in the 'retrieval_context'.",
            "Ensure that the 'actual_output' accurately reflects the information in the 'retrieval_context'. Slight vagueness or subjectivity is allowed if it does not introduce new or false facts.",
            "Minor omissions from the 'retrieval_context' can be tolerated, but the answer should not rely on unsupported or external information."
        ],
        evaluation_params=[
            LLMTestCaseParams.ACTUAL_OUTPUT,  # The model's generated answer
            LLMTestCaseParams.RETRIEVAL_CONTEXT  # The documents used to generate the answer
        ]
    )

  hallucination_detection_metric_test_case = LLMTestCase(
    input=row['input'],
    actual_output=row['actual_output'],
    retrieval_context=row['actual_context_to_llm'],

  )
  hallucination_detection_metric.measure(hallucination_detection_metric_test_case)

  return hallucination_detection_metric.score#, hallucination_detection_metric.reason


In [70]:
def calculate_hallucination_detection_metric(df_with_metrics):

  df_with_metrics['hallucination_detection_metric_score'] = df_with_metrics.apply(calculate_hallucination_detection, axis=1)
  return df_with_metrics

In [71]:
import math


def calculate_dcg(fractions):
    """
    Вычисляет Discounted Cumulative Gain (DCG) для списка значений fraction.
    """
    dcg = 0.0
    for i, rel in enumerate(fractions):
        dcg += rel / math.log2(i + 2)  # Позиции начинаются с 1, поэтому i+2
    return dcg


def calculate_ndcg(retrieved_fractions):
    """
    Вычисляет NDCG для каждого списка retrieved_fractions.

    Args:
        retrieved_fractions (list): Список списков значений fraction для каждого retrieved_chunk.

    Returns:
        list: Список значений NDCG для каждого списка.
    """
    ndcg_values = []

    for fractions in retrieved_fractions:
        # Рассчитываем DCG
        dcg = calculate_dcg(fractions)

        # Рассчитываем идеальный DCG (IDCG), сортируя fractions по убыванию
        ideal_fractions = sorted(fractions, reverse=True)
        idcg = calculate_dcg(ideal_fractions)

        # Рассчитываем NDCG
        if idcg == 0:
            ndcg_values.append(0.0)
        else:
            ndcg_values.append(dcg / idcg)

    return ndcg_values



In [72]:
def calculate_ndcg_metric(df_with_metrics):
  df_with_metrics['ndcg_metric_score'] = calculate_ndcg(df_with_metrics['is_relevant_ids_list'])
  return df_with_metrics

In [73]:
def calculate_recall(ground_truth_lists, float_lists):
    """
    Рассчитывает значение recall для каждого списка ground_truth.

    Args:
        ground_truth_lists (list): Список списков со строками ground_truth.
        float_lists (list): Список списков с флоатами, которые соответствуют значению метрик для каждого ground_truth.

    Returns:
        list: Список значений recall (float от 0 до 1) для каждого блока данных.
    """
    recall_values = []

    # Проходим по каждому списку ground_truth и соответствующему списку с float
    for ground_truth, float_list in zip(ground_truth_lists, float_lists):
        num_ground_truth = len(ground_truth)  # Количество строк в ground_truth

        if num_ground_truth == 0:
            recall_values.append(0.0)  # Если ground_truth пуст, recall = 0
        else:
            total_floats = sum(float_list)  # Суммируем все значения float
            recall = total_floats / num_ground_truth  # Делим сумму флоатов на количество строк ground_truth
            recall_values.append(min(recall, 1.0))  # Убеждаемся, что recall не превышает 1

    return recall_values



In [74]:
def calculate_recall_metric(df_with_metrics: pd.DataFrame):
  df_with_metrics['recall_metric_score'] = calculate_recall([[df_with_metrics.iloc[i]['id']] for i in range(len(df_with_metrics['id']))], df_with_metrics['is_relevant_ids_list'])

  return df_with_metrics

In [75]:
def calculate_metrics(df_with_answers: pd.DataFrame): #->
  df_with_metrics = df_with_answers.copy()
  df_with_metrics['is_relevant_ids_list'] = df_with_metrics.apply(lambda x: [0 if x['actual_context_id'][i] != x['id'] else 1 for i in range(len(x['actual_context_id']))], axis=1)

  df_with_metrics = calculate_relevance_metric(df_with_metrics)
  df_with_metrics = calculate_answer_relevance_metric(df_with_metrics)
  df_with_metrics = calculate_hallucination_detection_metric(df_with_metrics)

  df_with_metrics = calculate_ndcg_metric(df_with_metrics)
  df_with_metrics = calculate_recall_metric(df_with_metrics)


  return df_with_metrics



In [76]:
def create_dataset_for_metrics(llm_answers_info):
  return pd.DataFrame({'input': [llm_answer_info['question'] for llm_answer_info in llm_answers_info],
                       'actual_output': [llm_answer_info['answer'] for llm_answer_info in llm_answers_info],
                       'actual_context_to_llm': [format_docs(llm_answer_info['context']).split('\n\n') for llm_answer_info in llm_answers_info],
                       'actual_context_id': [[doc.metadata['id_'] for doc in llm_answer_info['context']] for llm_answer_info in llm_answers_info],
                       },
                      )

In [77]:
def run(settings: DefaultSettings):
  data_for_db = load_data(settings)
  db = create_vectorstore(data_for_db, settings)
  # retriever = create_retriever(db)

  chain = create_simple_chain_with_sources(db)

  questions_df = pd.read_parquet(settings.evaluation_df_path)

  questions_df = questions_df.head(5)

  questions_df_interim = questions_df.merge(data_for_db[['id', 'Вопрос из БЗ']], how='left', on=['Вопрос из БЗ'])
  questions = questions_df_interim['Вопрос пользователя'].to_list()

  llm_answers_info = generate_answers_invoke(questions, chain)
  eval_df = create_dataset_for_metrics(llm_answers_info)
  # display(eval_df)
  evaluation_df = pd.read_parquet(settings.evaluation_df_path)
  questions_df_interim = evaluation_df.merge(data_for_db[['id', 'Вопрос из БЗ']], how='left', on=['Вопрос из БЗ'])
  questions_df_interim.rename(columns={'Вопрос пользователя': 'input', 'Ответ сотрудника': 'ground_truth'}, inplace=True)
  for col in questions_df_interim.drop(columns=['Вопрос из БЗ']).columns:
    eval_df[col] = questions_df_interim[col]

  final_df = calculate_metrics(eval_df)
  return final_df

data_for_db = load_data(settings)
data_for_db

db = create_vectorstore(data_for_db, settings)
# retriever = create_retriever(db)

chain = create_simple_chain_with_sources(db)

questions_df = pd.read_parquet(settings.evaluation_df_path)
questions_df = questions_df.head(5)
questions_df

questions_df_interim = questions_df.merge(data_for_db[['id', 'Вопрос из БЗ']], how='left', on=['Вопрос из БЗ'])
questions = questions_df_interim['Вопрос пользователя'].to_list()

llm_answers_info = generate_answers_invoke(questions, chain)
llm_answers_info

eval_df = create_dataset_for_metrics(llm_answers_info)
evaluation_df = pd.read_parquet(settings.evaluation_df_path)
questions_df_interim = evaluation_df.merge(data_for_db[['id', 'Вопрос из БЗ']], how='left', on=['Вопрос из БЗ'])
questions_df_interim.rename(columns={'Вопрос пользователя': 'input', 'Ответ сотрудника': 'ground_truth'}, inplace=True)
for col in questions_df_interim.drop(columns=['Вопрос из БЗ']).columns:
  eval_df[col] = questions_df_interim[col]

final_df = calculate_metrics(eval_df)

final_df

In [78]:
df = pd.read_parquet('/content/sample_data/data/actual/interim/data_for_db.parquet')
df

,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
2,Почему могут отключить монетизацию из-за искус...,Монетизация на RUTUBE зависит в том числе от к...,МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,"Для каких статусов доступна монетизация, и поч...","Монетизацию на RUTUBE можно подключить, если в...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,Какой контент можно использовать для монетизац...,"То, что вы создали сами: видео, которое вы сня...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
...,...,...,...,...
320,"Видео недавно загрузилось, почему оно до сих п...","Все видео, которые вы загружаете на RUTUBE, пр...",МОДЕРАЦИЯ,Долгая модерация
322,Почему изменилась категория у моего видео?,Если при загрузке видео вы выбирали одну катег...,МОДЕРАЦИЯ,Смена категории/возрастные ограничения
323,Как удалить видео?,"Войдите в профиль на RUTUBE, перейдите в разде...",ВИДЕО,Загрузка видео
324,Как добавить тайм-коды под видео?,Перейдите в раздел «Видео» в Студии RUTUBE.\nН...,ВИДЕО,Загрузка видео


In [79]:
df2 = pd.read_parquet('/content/sample_data/data/actual/interim/evaluation_df.parquet')
df2

,Вопрос пользователя,Ответ сотрудника,Вопрос из БЗ
0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...,Что нельзя публиковать на RUTUBE?
1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...,Что нельзя публиковать на RUTUBE?
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...
3,Что запрещено в монетизации и что можно выклад...,"Здравствуйте!\nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...
4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Для монетизации можно использовать то, что вы ...",Почему могут отключить монетизацию из-за автор...
...,...,...,...
796,Добрый день! Как добавить таймкоды в видео?,Характеристики эпизодов:\n\nОпределяются по та...,Как добавить тайм-коды под видео?
797,Как добавить временные интервалы,Перейдите в раздел «Видео» в Студии RUTUBE.\nН...,Как добавить тайм-коды под видео?
798,В каком формате добавляются таймкоды?,Формат: минуты:секунды или часы:минуты:секунды...,В каком формате нужно добавлять тайм-коды под ...
799,Как мне правильно указать тайм-код?,Можете указать минуты:секунды или часы:минуты:...,В каком формате нужно добавлять тайм-коды под ...


In [80]:
df2.apply(lambda row: row['Вопрос пользователя'], axis=1)

,0
0,Здравствуйте! Можно уточнить причины Правилhtt...
1,"Добрый вечер, какой топ причин блокировки виде..."
2,"Все пишут, что монетизация на рутубе отключает..."
3,Что запрещено в монетизации и что можно выклад...
4,"Чтобы не отключали монетизацию, надо, чтобы я ..."
...,...
796,Добрый день! Как добавить таймкоды в видео?
797,Как добавить временные интервалы
798,В каком формате добавляются таймкоды?
799,Как мне правильно указать тайм-код?


In [81]:
res = run(settings)

<ipython-input-17-968c83cb0021>:11: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client(
INFO:weaviate-client:embedded weaviate is already listening on port 8079


Number of documents in database: {'data': {'Aggregate': {'Rutube': [{'meta': {'count': 0}}]}}}
287


,input,actual_output,actual_context_to_llm,actual_context_id
0,Здравствуйте! Можно уточнить причины Правилhtt...,По ссылке https://rutube.ru/info/taboo_agreeme...,"[Я нашел на RUTUBE ролик, нарушающий ваши прав...","[295, 320, 319, 231, 39, 279, 144, 149, 152, 232]"
1,"Добрый вечер, какой топ причин блокировки виде...",Блокировка видео на Rutube может происходить п...,"[Видео недавно загрузилось, почему оно до сих ...","[320, 319, 279, 136, 152, 295, 278, 231, 39, 1]"
2,"Все пишут, что монетизация на рутубе отключает...",Монетизация на Rutube может быть отключена всл...,[Что такое Монетизация на RUTUBE? Монетизацию ...,"[193, 1, 3, 2, 216, 196, 211, 210, 149, 209]"
3,Что запрещено в монетизации и что можно выклад...,Монетизация запрещает использование чужого кон...,"[Для каких статусов доступна монетизация, и по...","[3, 4, 1, 0, 2, 6, 213, 208, 197, 194]"
4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Из контекста следует, что для того, чтобы не о...",[Почему могут отключить монетизацию из-за авто...,"[1, 2, 3, 190, 210, 211, 196, 216, 212, 275]"


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

...

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

In [82]:
res

,input,actual_output,actual_context_to_llm,actual_context_id,ground_truth,id,is_relevant_ids_list,relevance_metric_score,answer_relevance_metric_score,hallucination_detection_metric_score,ndcg_metric_score,recall_metric_score
0,Здравствуйте! Можно уточнить причины Правилhtt...,По ссылке https://rutube.ru/info/taboo_agreeme...,"[Я нашел на RUTUBE ролик, нарушающий ваши прав...","[295, 320, 319, 231, 39, 279, 144, 149, 152, 232]",Добрый день!\nЧто нельзя публиковать на RUTUBE...,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.0,0.0,0.0,0.00000,0.0
1,"Добрый вечер, какой топ причин блокировки виде...",Блокировка видео на Rutube может происходить п...,"[Видео недавно загрузилось, почему оно до сих ...","[320, 319, 279, 136, 152, 295, 278, 231, 39, 1]",Добрый вечер!\nЧто заперщено публиковать на RU...,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.0,1.0,0.0,0.00000,0.0
2,"Все пишут, что монетизация на рутубе отключает...",Монетизация на Rutube может быть отключена всл...,[Что такое Монетизация на RUTUBE? Монетизацию ...,"[193, 1, 3, 2, 216, 196, 211, 210, 149, 209]","Добрый день! \nМонетизация может отключиться, ...",1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",0.0,0.0,0.0,0.63093,1.0
3,Что запрещено в монетизации и что можно выклад...,Монетизация запрещает использование чужого кон...,"[Для каких статусов доступна монетизация, и по...","[3, 4, 1, 0, 2, 6, 213, 208, 197, 194]","Здравствуйте!\nМонетизация может отключиться, ...",1,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",1.0,1.0,1.0,0.50000,1.0
4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Из контекста следует, что для того, чтобы не о...",[Почему могут отключить монетизацию из-за авто...,"[1, 2, 3, 190, 210, 211, 196, 216, 212, 275]","Для монетизации можно использовать то, что вы ...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.0,0.0,1.0,1.00000,1.0
